In [1]:
import tensorflow as tf
import numpy as np

C:\Users\haho6\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [220]:
from keras.preprocessing.image import load_img, img_to_array
images = []
for i in range(470):
    image = load_img('./DATA/'+str(i)+'.jpeg',target_size=(64,64))
    image = img_to_array(image)
    image.shape
    image = image / 256
    images.append(image)
    
    
images = np.array(images)

In [3]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-3., 3., size = [batch_size, n_noise])

In [4]:
def windowUp(arr):
    import cv2
    frame = cv2.cvtColor(np.array(arr), cv2.COLOR_BGR2RGB)
    cv2.imshow("test", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [221]:
X = tf.placeholder(tf.float32, [None, 64,64,3])
Y = tf.placeholder(tf.float32, [None, 1])
Z = tf.placeholder(tf.float32, [None, 100])

In [222]:
def generate(Z):
    w1 = tf.Variable(tf.random_normal([100, 16384]))
    h1 = tf.matmul(Z,w1)
    re = tf.reshape(h1,[-1,4,4,1024])
    re = tf.layers.batch_normalization(re)

    conv_h1 = tf.layers.conv2d_transpose(re ,512, 5, 2, padding='same')
    conv_h1 = tf.nn.relu(conv_h1)

    conv_h2 = tf.layers.conv2d_transpose(conv_h1 ,256, 4, 2, padding='same')
    conv_h2 = tf.nn.relu(conv_h2)

    conv_h3 = tf.layers.conv2d_transpose(conv_h2 ,128, 4, 2, padding='same')
    conv_h3 = tf.nn.relu(conv_h3)
    
#    conv_h4 = tf.layers.conv2d_transpose(conv_h3 ,64, 4, 2, padding='same')
#    conv_h4 = tf.nn.relu(conv_h4)
    
    conv_h6 = tf.layers.conv2d_transpose(conv_h3 ,3, 4, 2, padding='same')
    return conv_h6

In [269]:
w1 = tf.Variable(tf.random_normal([100, 16384]))
h1 = tf.matmul(Z,w1)
re = tf.reshape(h1,[-1,4,4,1024])

W1 = tf.Variable(tf.random_normal([3,3,512,1024],stddev=0.02))
tf.nn.conv2d_transpose(re,W1,[-1,8,8,512],[1,2,2,1])

<tf.Tensor 'conv2d_transpose_31:0' shape=(?, 8, 8, 512) dtype=float32>

In [252]:
def discriminaster(inputs):
        W1 = tf.Variable(tf.random_normal([3,3,3,32],stddev=0.02))
        H1 = tf.nn.conv2d(inputs,W1, strides=[1,2,2,1], padding="SAME")
        H1 = tf.layers.batch_normalization(H1)
        H1 = tf.nn.relu(H1)

        W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.02))
        H2 = tf.nn.conv2d(H1,W2, strides=[1,2,2,1], padding="SAME")
        H2 = tf.nn.relu(H2)

        W3 = tf.Variable(tf.random_normal([3,3,64,256],stddev=0.02))
        H3 = tf.nn.conv2d(H2,W3, strides=[1,2,2,1], padding="SAME")
        H3 = tf.nn.relu(H3)

        out = tf.reshape(H3,[-1,16384])
        out = tf.layers.dense(out,1)
        return out

In [253]:
G = generate(Z)
D_real = discriminaster(X)
D_gene = discriminaster(G)

In [254]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real,labels=tf.ones_like(D_real)))

loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene,labels=tf.zeros_like(D_gene)))

In [255]:
loss_D = loss_D_gene + loss_D_real
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [256]:
train_D = tf.train.AdamOptimizer(0.01).minimize(loss_D)
train_G = tf.train.AdamOptimizer(0.01).minimize(loss_G)

In [257]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [262]:
im = images[255:260]

for i in range(40):
    noise = get_noise(5,100)
    _, varsD=sess.run([train_D, loss_D], 
                  feed_dict={
                      X:im,
                      Z: noise})
    
    _, varsG=sess.run([train_G, loss_G], 
                  feed_dict={
                      X:im,
                      Z: noise})
    print(str(i)+"   D : "+str(varsD)+"\t G : "+str(varsG))


0   D : 0.6486298	 G : 3.2939365
1   D : 0.65147424	 G : 2.6533058
2   D : 0.43201622	 G : 5.7126656
3   D : 0.069326654	 G : 10.02589
4   D : 0.0105508715	 G : 34.299263
5   D : 0.004176916	 G : 17.452185
6   D : 0.0008604784	 G : 19.234522
7   D : 0.0016715571	 G : 16.487425
8   D : 0.015580607	 G : 12.220519
9   D : 0.11295734	 G : 10.742517
10   D : 0.2966604	 G : 11.388365
11   D : 0.3048776	 G : 206.89926
12   D : 0.13216904	 G : 19.115206
13   D : 0.018097812	 G : 57.993843
14   D : 0.006788359	 G : 84.99779
15   D : 0.019014174	 G : 22.051512
16   D : 0.020542046	 G : 129.85736
17   D : 0.004793178	 G : 31.62359
18   D : 1.8302882e-05	 G : 15.694987
19   D : 0.0005245249	 G : 10.333112
20   D : 0.03671009	 G : 21.969425
21   D : 0.07331268	 G : 31.757122
22   D : 0.042707067	 G : 19.56508
23   D : 0.016917208	 G : 19.553322
24   D : 0.007871864	 G : 16.494305
25   D : 0.004592662	 G : 23.973806
26   D : 0.0007774404	 G : 27.958271
27   D : 0.0020710465	 G : 26.849564
28   D : 0

In [263]:

noise = get_noise(20,100)
i = sess.run(G, feed_dict={Z:noise})
for row in range(20):
    windowUp(i[row])

In [388]:
i.shape

(1, 64, 64, 3)

In [207]:
noise = get_noise(1,100)